In [1]:
import csv
import ast
import numpy as np
import time
import random

start = time.time()

# --- Load Billboard Data (BB) ---
BB = []
with open("BB_LA.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        BB.append({
            'idx': int(row['B_index']),
            'cost': float(row['Cost'])
        })

# --- Load User Influence Data (ub) ---
ub = []
with open("new_ub.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        row['Influenced Billboards'] = ast.literal_eval(row['Influenced Billboards'])
        ub.append(row)

# --- Load Demand and Budget (from advertiser_1.csv) ---
Dem = []
Budget = []
with open("Advertiser_LA1.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        Dem.append(float(row['Demand']))
        Budget.append(float(row['Payment']))

print("Budgets:", Budget)

# --- BillboardSlot Class ---
class BillboardSlot:
    def __init__(self, idx):
        self.idx = idx

    def __repr__(self):
        return f"Slot({self.idx})"

# --- Convert BB to BillboardSlot Objects ---
BS = [BillboardSlot(item['idx']) for item in BB]


Budgets: [1758.0, 1318.0, 1305.0, 1196.0, 1626.0, 1515.0, 1297.0, 1187.0, 1184.0, 1506.0, 1398.0, 1395.0, 1502.0, 1178.0, 1285.0, 1494.0, 1490.0, 1277.0, 1274.0, 1270.0, 1691.0, 1475.0, 1474.0, 1263.0, 1367.0, 1472.0, 1575.0, 1154.0, 1154.0, 1571.0, 1465.0, 1360.0, 1565.0, 1458.0, 1245.0, 1444.0, 1131.0, 1336.0, 1336.0, 1130.0, 1130.0, 1639.0, 1228.0, 1226.0, 1225.0, 1429.0, 1630.0, 1120.0, 1322.0, 1423.0, 1524.0, 1117.0, 1418.0, 1517.0, 1211.0, 1310.0, 1511.0, 1509.0, 1202.0, 1498.0, 1097.0, 1493.0, 1094.0, 1392.0, 1193.0, 1092.0, 1587.0, 1483.0, 1184.0, 1181.0, 1371.0, 1077.0, 1562.0, 1072.0, 1363.0, 1549.0, 1060.0, 1346.0, 1344.0, 1051.0, 1145.0, 1142.0, 1139.0, 1423.0, 1042.0, 1041.0, 1318.0, 1502.0, 1403.0, 1306.0, 1119.0, 1392.0, 1291.0, 1287.0, 1469.0, 1368.0, 999.0, 1268.0, 994.0, 1352.0]


In [2]:
# Influence function
def influence(S):
    influence_score = 0.0
    slot_ids = {s.idx if isinstance(s, BillboardSlot) else s for s in S}
   
    for row in ub:
        influenced_billboards = row.get('Influenced Billboards', {})
       
        intersection_result = {int(key): value for key, value in influenced_billboards.items() if int(key) in slot_ids}
       
        product = 1.0
        for value in intersection_result.values():
            product *= (1 - value)
        influence_score += (1 - product)
    return influence_score

In [3]:
def random_pick(BS, Dem, Budget):
    import random

    # Build a mapping from slot.idx to cost
    slot_cost_map = {slot['idx']: slot['cost'] for slot in BB}

    available_slots = {slot.idx for slot in BS}
    used_slots = set()
    Si_list = [set() for _ in Dem]     # Slot sets per product
    total_costs = [0.0 for _ in Dem]   # Total cost per product

    for j in range(len(Dem)):
        demand = Dem[j]
        remaining_budget = Budget[j]
        current_influence = 0.0

        while current_influence < demand:
            remaining = list(available_slots - used_slots - Si_list[j])
            if not remaining:
                print(f"No more slots available to fulfill demand for product {j}.")
                break

            selected = random.choice(remaining)
            selected_cost = slot_cost_map.get(selected, 0)

            if selected_cost > remaining_budget:
                continue  # Skip if this slot exceeds remaining budget

            # Accept the slot
            Si_list[j].add(selected)
            total_costs[j] += selected_cost
            remaining_budget -= selected_cost
            current_influence = influence(Si_list[j])
            used_slots.add(selected)

        if current_influence < demand:
            print(f"Warning: Demand for product {j} not fully satisfied (Influence = {current_influence:.2f}, Needed = {demand:.2f}).")

    return Si_list


In [4]:
# Run the allocation
result = random_pick(BS, Dem, Budget)
total_influence = 0.0

# Print per-product results with Demand
for i, slots in enumerate(result):
    slot_cost = sum(BB[slot]['cost'] for slot in slots if slot < len(BB))
    achieved_influence = influence(slots)
    total_influence += achieved_influence
    # print(f"Product {i+1}: Demand = {Dem[i]:.2f}, Influence = {achieved_influence:.2f}, "
    #       f"Cost = {slot_cost:.2f}, Budget = {Budget[i]:.2f}, Slots = {sorted(list(slots))}")

# Total unique slots and influence
all_selected = set().union(*result)
print("\nTotal Unique Slots Selected:", len(all_selected))
print("Total Combined Influence:",total_influence)
print("Execution Time:", round(time.time() - start, 2), "seconds")

# Count satisfied products
satisfied_count = sum(1 for j in range(len(Dem)) if influence(result[j]) >= Dem[j])
print("Number of products whose demands are satisfied:", satisfied_count, "out of", len(Dem))



Total Unique Slots Selected: 1272
Total Combined Influence: 1461.8853945223923
Execution Time: 3.18 seconds
Number of products whose demands are satisfied: 100 out of 100


In [5]:
h = len(Dem)
count = 0

for j in range(h):
    if influence(result[j]) >= Dem[j]:
        count += 1

print("Number of products whose demands are satisfied:", count, "out of", h)


Number of products whose demands are satisfied: 100 out of 100
